In [34]:
#Prueba de actualizacion - Base de dato DNRPA
# Carga de librerias

import datetime
import mysql.connector
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import openpyxl
import pandas as pd
from datetime import datetime
import os

#Variables globales
host = '172.17.22.10'
user = 'Ivan'
password = 'Estadistica123'
database = 'prueba1'

In [35]:
# Conexion con BDD 
conn = mysql.connector.connect( host=host, user=user, password=password, database=database)

#Instancia manejadora de BDD

cursor = conn.cursor()

In [36]:
#Busqueda por nombre de TABLA 

table_name = "dnrpa_inscripcion_nacion_auto"

#Creacion de orden en lenguaje MYSQL
select_dates_query = "SELECT * FROM dnrpa_inscripcion_corrientes_auto"

#Obtencion del DATAFRAME - uso de Orden creada y Conexion.
df = pd.read_sql(select_dates_query,con=conn)

#Busqueda del año mas grande (actual o anterior) --> Aclaracion por posible conflicto en los avisos por cada año nuevo
año_de_comparacion = int(df['Fecha'].max().year)



C:\Users\Administrador\AppData\Local\Temp\ipykernel_7616\2905415087.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(select_dates_query,con=conn)


In [39]:
from pathlib import Path
import os

# Obtener el directorio actual del notebook
directorio_actual = Path(os.getcwd())

# Obtener la ruta completa del archivo .ipynb
ruta_archivo_ipynb = directorio_actual.joinpath(Path())

# Obtener la dirección del archivo .ipynb
direccion_archivo_ipynb = str(ruta_archivo_ipynb.resolve())

# Mostrar la dirección del archivo
print(direccion_archivo_ipynb)

C:\Users\Administrador\Desktop\scrapingTrabajo\script_DNRPA


In [51]:
#===== Obtencion de datos de la web DROPNA =====#

try:
# Obtener la ruta del directorio actual (donde se encuentra el script)
    directorio_actual = direccion_archivo_ipynb
    ruta_carpeta_files = os.path.join(directorio_actual, 'files')
    ruta_archivo_excel = os.path.join(ruta_carpeta_files, 'registroInscripcionAutoNacion.xlsx')

            #↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓ SELENIUM ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
    driver = webdriver.Chrome()
    driver.get('https://www.dnrpa.gov.ar/portal_dnrpa/estadisticas/rrss_tramites/tram_prov.php?origen=portal_dnrpa&tipo_consulta=inscripciones')
            # Obtener la ventana actual
    ventana_actual = driver.current_window_handle
            
    elemento = driver.find_element(By.XPATH, '//*[@id="seleccion"]/center/table/tbody/tr[2]/td/select')
    # Obtener todas las opciones del elemento select
    opciones = elemento.find_elements(By.TAG_NAME, 'option')

    # Buscar la opción deseada por su valor y hacer clic en ella
    valor_deseado = '2023' # Valor de la opción que deseas seleccionar

    for opcion in opciones:
        if opcion.get_attribute('value') == valor_deseado:
            opcion.click()
            break
    
    boton = driver.find_element(By.XPATH, '//*[@id="seleccion"]/center/table/tbody/tr[4]/td/input[1]')
    boton.click()
    
    time.sleep(5)
    
    boton_aceptar = driver.find_element(By.XPATH, '//*[@id="seleccion"]/center/center/input')
    boton_aceptar.click()
    
    # Esperar un momento para que se abra la nueva pestaña
    driver.implicitly_wait(5)
    # Cambiar al contexto de la nueva pestaña
    for ventana in driver.window_handles:
        if ventana != ventana_actual:
            driver.switch_to.window(ventana)
    
    time.sleep(5)
    #↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓ ENCONTRAR Y TOMAR LOS DATOS ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
    # Encontrar el elemento <div> con la clase 'grid'
    elemento_div = driver.find_element(By.CLASS_NAME, 'grid')

    # Encontrar la tabla dentro del elemento <div>
    elemento_tabla = elemento_div.find_element(By.TAG_NAME, 'table')

    # Obtener todas las filas de la tabla
    filas = elemento_tabla.find_elements(By.TAG_NAME, 'tr')

    # Lista para almacenar los datos de la tabla
    tabla_datos = []
    
    for fila in filas:
        # Obtener las celdas de cada fila, excluyendo la última columna y la última celda de encabezado
        celdas = fila.find_elements(By.TAG_NAME, 'th') + fila.find_elements(By.TAG_NAME, 'td')[:-1]

        # Lista para almacenar los valores de cada fila
        fila_datos = []

        for celda in celdas:
            valor = celda.text
            if isinstance(valor, str):
                # Verificar si el valor comienza con un número
                if valor.strip() and valor[0].isdigit():
                    try:
                        # Reemplazar el punto decimal por una coma (si es necesario)
                        valor = valor.replace('.', '')
                        # Intentar convertir el valor a float
                        valor = float(valor)
                        print("valor1: ", valor)
                    except ValueError:
                        pass  # Mantener el valor original si no se puede convertir a float
            fila_datos.append(valor)
        print("aca: ", fila_datos)
        # Verificar si la última celda es "Total" y eliminarla
        if fila_datos and fila_datos[-1] == "Total":
            fila_datos.pop()


        tabla_datos.append(fila_datos) 
    
    datos_sin_segunda_fila = tabla_datos[0:1] + tabla_datos[2:]
    # Transponer los datos utilizando pandas
    df = pd.DataFrame(datos_sin_segunda_fila)
    df_transpuesta = df.transpose()
    
    #Conversion de MESES a formato Y-M-D , tipo de dato: datetime
    print(df_transpuesta[0][1:])
    meses = df_transpuesta[0][1:]

    #Donde almacenamos las nuevas fechas
    nuevas_fechas = list()

    for i in range(1, len(meses)+ 1):

        if i < 10:
            fecha_str =  '01-0'+str(i)+"-"+ str(valor_deseado)
        else:
            fecha_str = '01-'+str(i)+"-"+ str(valor_deseado)

        fecha_str = datetime.strptime(fecha_str,'%d-%m-%Y').date()
        nuevas_fechas.append(fecha_str)

    #Reasignacion de fechas
    df_transpuesta[0][1:] = nuevas_fechas

except Exception as e:
            # Manejar cualquier excepción ocurrida durante la carga de datos
            print(f"Registro automotor: Ocurrió un error durante la carga de datos: {str(e)}")
            conn.close()  # Cerrar la conexión en caso de error

SyntaxError: '(' was never closed (792319251.py, line 22)